In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway
import sys
import os
sys.path.append(os.path.abspath('../data'))

In [5]:
file_path = 'C:\\Users\\King Betse\\Downloads\\MachineLearningRating_v3\\MachineLearningRating_v3.txt'

In [6]:
df = pd.read_csv(file_path, sep='|')

C:\Users\King Betse\AppData\Local\Temp\ipykernel_20300\1248705775.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='|')


## Checking data quality

In [7]:
df = pd.read_csv(file_path, sep='|')

C:\Users\King Betse\AppData\Local\Temp\ipykernel_20300\1248705775.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='|')


In [8]:
# Check for duplicates
print(f'Duplicates in the data: {df.duplicated().sum()}')

Duplicates in the data: 0


## Select Metrics KPI 

In [9]:
# Calculate Loss Ratio
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']

# Calculate Profit Margin
df['ProfitMargin'] = (df['TotalPremium'] - df['TotalClaims']) / df['TotalPremium']

# Assuming that the number of claims is derived from the `TotalClaims` and `PolicyID` columns (one claim per policy)
# or if there's a specific column for claims, replace accordingly.
df['ClaimFrequency'] = df['TotalClaims'] / df['PolicyID'].nunique()

# Display the first few rows to check the calculated KPIs
df[['LossRatio', 'ProfitMargin', 'ClaimFrequency']].head()


,LossRatio,ProfitMargin,ClaimFrequency
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,NaN,NaN,0.0
3,0.0,1.0,0.0
4,NaN,NaN,0.0


In [10]:
# Group the data by Province and calculate the mean of KPIs
province_group = df.groupby('Province')[['LossRatio', 'ProfitMargin', 'ClaimFrequency']].mean()

# Display summary statistics by Province
province_group


,LossRatio,ProfitMargin,ClaimFrequency
Province,,,
Eastern Cape,inf,-inf,0.006388
Free State,inf,-inf,0.006260
Gauteng,NaN,NaN,0.010661
KwaZulu-Natal,inf,-inf,0.012033
Limpopo,inf,-inf,0.005847
Mpumalanga,NaN,NaN,0.005541
North West,inf,-inf,0.005902
Northern Cape,0.203831,0.796169,0.002004
Western Cape,inf,-inf,0.008690


In [11]:
# Group the data by Zip Code and calculate the mean of KPIs
zip_code_group = df.groupby('PostalCode')[['LossRatio', 'ProfitMargin', 'ClaimFrequency']].mean()

# Display summary statistics by Zip Code
zip_code_group


,LossRatio,ProfitMargin,ClaimFrequency
PostalCode,,,
1,inf,-inf,0.008227
2,0.852758,0.147242,0.005941
4,0.000000,1.000000,0.000000
5,0.698901,0.301099,0.029626
6,0.122927,0.877073,0.002801
...,...,...,...
9781,inf,-inf,0.019929
9830,0.000000,1.000000,0.000000
9868,0.000000,1.000000,0.000000


In [12]:
# Group the data by Gender and calculate the mean of KPIs
gender_group = df.groupby('Gender')[['LossRatio', 'ProfitMargin', 'ClaimFrequency']].mean()

# Display summary statistics by Gender
gender_group


,LossRatio,ProfitMargin,ClaimFrequency
Gender,,,
Female,inf,-inf,0.005292
Male,inf,-inf,0.004660
Not specified,NaN,NaN,0.009521


In [13]:
from scipy.stats import f_oneway

# Perform ANOVA for Loss Ratio across Provinces
anova_loss_ratio_province = f_oneway(*[group['LossRatio'] for name, group in df.groupby('Province')])

# Perform ANOVA for Profit Margin across Provinces
anova_profit_margin_province = f_oneway(*[group['ProfitMargin'] for name, group in df.groupby('Province')])

# Perform ANOVA for Claim Frequency across Provinces
anova_claim_freq_province = f_oneway(*[group['ClaimFrequency'] for name, group in df.groupby('Province')])

# Print ANOVA results
print(f"ANOVA Results for Loss Ratio (Provinces): F-statistic = {anova_loss_ratio_province.statistic}, p-value = {anova_loss_ratio_province.pvalue}")
print(f"ANOVA Results for Profit Margin (Provinces): F-statistic = {anova_profit_margin_province.statistic}, p-value = {anova_profit_margin_province.pvalue}")
print(f"ANOVA Results for Claim Frequency (Provinces): F-statistic = {anova_claim_freq_province.statistic}, p-value = {anova_claim_freq_province.pvalue}")


ANOVA Results for Loss Ratio (Provinces): F-statistic = nan, p-value = nan
ANOVA Results for Profit Margin (Provinces): F-statistic = nan, p-value = nan
ANOVA Results for Claim Frequency (Provinces): F-statistic = 5.849413762407612, p-value = 1.6782057588675906e-07


In [14]:
from scipy.stats import ttest_ind

# Perform t-test for Loss Ratio between Men and Women
t_test_loss_ratio_gender = ttest_ind(df[df['Gender'] == 'Male']['LossRatio'],
                                     df[df['Gender'] == 'Female']['LossRatio'])

# Perform t-test for Profit Margin between Men and Women
t_test_profit_margin_gender = ttest_ind(df[df['Gender'] == 'Male']['ProfitMargin'],
                                        df[df['Gender'] == 'Female']['ProfitMargin'])

# Perform t-test for Claim Frequency between Men and Women
t_test_claim_freq_gender = ttest_ind(df[df['Gender'] == 'Male']['ClaimFrequency'],
                                     df[df['Gender'] == 'Female']['ClaimFrequency'])

# Print t-test results
print(f"T-test Results for Loss Ratio (Gender): t-statistic = {t_test_loss_ratio_gender.statistic}, p-value = {t_test_loss_ratio_gender.pvalue}")
print(f"T-test Results for Profit Margin (Gender): t-statistic = {t_test_profit_margin_gender.statistic}, p-value = {t_test_profit_margin_gender.pvalue}")
print(f"T-test Results for Claim Frequency (Gender): t-statistic = {t_test_claim_freq_gender.statistic}, p-value = {t_test_claim_freq_gender.pvalue}")


T-test Results for Loss Ratio (Gender): t-statistic = nan, p-value = nan
T-test Results for Profit Margin (Gender): t-statistic = nan, p-value = nan
T-test Results for Claim Frequency (Gender): t-statistic = -0.24803623812388736, p-value = 0.8041073961270342


In [15]:
from scipy import stats

# Perform ANOVA for Loss Ratio by Province
anova_province_loss_ratio = stats.f_oneway(*[group['LossRatio'].values for name, group in df.groupby('Province')])

# Print p-value for ANOVA test
print(f"ANOVA for Loss Ratio across Provinces: p-value = {anova_province_loss_ratio.pvalue}")

# Check if p-value is significant
if anova_province_loss_ratio.pvalue < 0.05:
    print("Reject the null hypothesis: There is a significant difference in risk across provinces.")
else:
    print("Fail to reject the null hypothesis: No significant difference in risk across provinces.")


ANOVA for Loss Ratio across Provinces: p-value = nan
Fail to reject the null hypothesis: No significant difference in risk across provinces.


In [16]:
# Perform ANOVA for Profit Margin by Zip Code
anova_zipcode_profit_margin = stats.f_oneway(*[group['ProfitMargin'].values for name, group in df.groupby('PostalCode')])

# Print p-value for ANOVA test
print(f"ANOVA for Profit Margin across Zip Codes: p-value = {anova_zipcode_profit_margin.pvalue}")

# Check if p-value is significant
if anova_zipcode_profit_margin.pvalue < 0.05:
    print("Reject the null hypothesis: There is a significant margin difference across zip codes.")
else:
    print("Fail to reject the null hypothesis: No significant margin difference across zip codes.")


ANOVA for Profit Margin across Zip Codes: p-value = nan
Fail to reject the null hypothesis: No significant margin difference across zip codes.


In [17]:
# Group data by gender
male_group = df[df['Gender'] == 'Male']
female_group = df[df['Gender'] == 'Female']

# Perform T-test for Loss Ratio between Genders
t_test_gender_loss_ratio = stats.ttest_ind(male_group['LossRatio'], female_group['LossRatio'])

# Print p-value for T-test
print(f"T-test for Loss Ratio between Genders: p-value = {t_test_gender_loss_ratio.pvalue}")

# Check if p-value is significant
if t_test_gender_loss_ratio.pvalue < 0.05:
    print("Reject the null hypothesis: There is a significant difference in risk between Men and Women.")
else:
    print("Fail to reject the null hypothesis: No significant difference in risk between Men and Women.")


T-test for Loss Ratio between Genders: p-value = nan
Fail to reject the null hypothesis: No significant difference in risk between Men and Women.


In [18]:
# Perform T-test for Profit Margin between Genders
t_test_gender_profit_margin = stats.ttest_ind(male_group['ProfitMargin'], female_group['ProfitMargin'])

# Print p-value for T-test
print(f"T-test for Profit Margin between Genders: p-value = {t_test_gender_profit_margin.pvalue}")

# Check if p-value is significant
if t_test_gender_profit_margin.pvalue < 0.05:
    print("Reject the null hypothesis: There is a significant difference in profit margin between Men and Women.")
else:
    print("Fail to reject the null hypothesis: No significant difference in profit margin between Men and Women.")


T-test for Profit Margin between Genders: p-value = nan
Fail to reject the null hypothesis: No significant difference in profit margin between Men and Women.


In [19]:
# Display the first few rows to check the calculated KPIs
df[['LossRatio', 'ProfitMargin', 'ClaimFrequency']].head()


,LossRatio,ProfitMargin,ClaimFrequency
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,NaN,NaN,0.0
3,0.0,1.0,0.0
4,NaN,NaN,0.0


## Data Segmentation

In [20]:
# Code segmentation for the AlarmImmobiliser
# Group A: Plans without an alarm immobiliser
group_a_alarm = df[df['AlarmImmobiliser'] == 'No']

# Group B: Plans with an alarm immobiliser
group_b_alarm = df[df['AlarmImmobiliser'] == 'Yes']


In [21]:
# Working on the Tracking device
# Group A: Plans without a tracking device
group_a_tracking = df[df['TrackingDevice'] == 'No']

# Group B: Plans with a tracking device
group_b_tracking = df[df['TrackingDevice'] == 'Yes']

print(group_a_tracking, group_b_tracking)


         UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0                     145249     12827  2015-03-01 00:00:00             True   
1                     145249     12827  2015-05-01 00:00:00             True   
2                     145249     12827  2015-07-01 00:00:00             True   
3                     145255     12827  2015-05-01 00:00:00             True   
4                     145255     12827  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:00            False   

        Citizenship          LegalType 

In [22]:
# Working with NewVehicle
# Group A: Plans for non-new vehicles
group_a_new_vehicle = df[df['NewVehicle'] == 'No']

# Group B: Plans for new vehicles
group_b_new_vehicle = df[df['NewVehicle'] == 'Yes']

print(group_a_new_vehicle, group_b_new_vehicle)


Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCode, MainCrestaZone, SubCrestaZone, ItemType, mmcode, VehicleType, RegistrationYear, make, Model, Cylinders, cubiccapacity, kilowatts, bodytype, NumberOfDoors, VehicleIntroDate, CustomValueEstimate, AlarmImmobiliser, TrackingDevice, CapitalOutstanding, NewVehicle, WrittenOff, Rebuilt, Converted, CrossBorder, NumberOfVehiclesInFleet, SumInsured, TermFrequency, CalculatedPremiumPerTerm, ExcessSelected, CoverCategory, CoverType, CoverGroup, Section, Product, StatutoryClass, StatutoryRiskType, TotalPremium, TotalClaims, LossRatio, ProfitMargin, ClaimFrequency]
Index: []

[0 rows x 55 columns] Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCod

In [23]:
# Working with WrittenOff
# Group A: Plans where vehicles are not written off
group_a_written_off = df[df['WrittenOff'] == 'No']

# Group B: Plans where vehicles are written off
group_b_written_off = df[df['WrittenOff'] == 'Yes']

print(group_a_written_off, group_b_written_off)


         UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
12                    127439     10874  2015-02-01 00:00:00             True   
13                    127445     10874  2014-12-01 00:00:00             True   
14                    127445     10874  2015-03-01 00:00:00             True   
15                    127445     10874  2015-05-01 00:00:00             True   
16                    127445     10874  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:00            False   

        Citizenship          LegalType 

In [24]:
# Working with rebuilt
# Group A: Plans where vehicles are not rebuilt
group_a_rebuilt = df[df['Rebuilt'] == 'No']

# Group B: Plans where vehicles are rebuilt
group_b_rebuilt = df[df['Rebuilt'] == 'Yes']

print(group_a_rebuilt, group_b_rebuilt)


         UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
12                    127439     10874  2015-02-01 00:00:00             True   
13                    127445     10874  2014-12-01 00:00:00             True   
14                    127445     10874  2015-03-01 00:00:00             True   
15                    127445     10874  2015-05-01 00:00:00             True   
16                    127445     10874  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:00            False   

        Citizenship          LegalType 

In [25]:
# Working with converted
# Group A: Plans where vehicles are not converted
group_a_converted = df[df['Converted'] == 'No']

# Group B: Plans where vehicles are converted
group_b_converted = df[df['Converted'] == 'Yes']

print(group_a_converted, group_b_converted)


         UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
12                    127439     10874  2015-02-01 00:00:00             True   
13                    127445     10874  2014-12-01 00:00:00             True   
14                    127445     10874  2015-03-01 00:00:00             True   
15                    127445     10874  2015-05-01 00:00:00             True   
16                    127445     10874  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:00            False   

        Citizenship          LegalType 

In [26]:
# Working with cross-border
# Group A: Plans not covering cross-border usage
group_a_cross_border = df[df['CrossBorder'] == 'No']

# Group B: Plans covering cross-border usage
group_b_cross_border = df[df['CrossBorder'] == 'Yes']

print(group_a_cross_border, group_b_cross_border)


        UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
830715                  888       135  2014-02-01 00:00:00            False   
830716                  888       135  2014-09-01 00:00:00            False   
830717                  873       134  2013-12-01 00:00:00            False   
830718                  873       134  2014-03-01 00:00:00            False   
830719                  873       134  2014-05-01 00:00:00            False   
...                     ...       ...                  ...              ...   
973062                  989       142  2014-10-01 00:00:00            False   
973063                  989       142  2014-12-01 00:00:00            False   
973064                  989       142  2015-03-01 00:00:00            False   
973065                  989       142  2015-05-01 00:00:00            False   
973066                  989       142  2015-07-01 00:00:00            False   

       Citizenship        LegalType Title Language 

In [27]:
# Working with Cover Type
# Group A: Control Group ( Comprehensive Cover)
group_a = df[df['CoverType'] == 'Comprehensive']

# Group B: Test Group ( Third Party Cover)
group_b = df[df['CoverType'] == 'Third Party']

print(group_a, group_b)


Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCode, MainCrestaZone, SubCrestaZone, ItemType, mmcode, VehicleType, RegistrationYear, make, Model, Cylinders, cubiccapacity, kilowatts, bodytype, NumberOfDoors, VehicleIntroDate, CustomValueEstimate, AlarmImmobiliser, TrackingDevice, CapitalOutstanding, NewVehicle, WrittenOff, Rebuilt, Converted, CrossBorder, NumberOfVehiclesInFleet, SumInsured, TermFrequency, CalculatedPremiumPerTerm, ExcessSelected, CoverCategory, CoverType, CoverGroup, Section, Product, StatutoryClass, StatutoryRiskType, TotalPremium, TotalClaims, LossRatio, ProfitMargin, ClaimFrequency]
Index: []

[0 rows x 55 columns]          UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
5                     145247     12827  2015-01-01 00:00:00             True   
6                     145247     12827  201

In [28]:
# Working with language
# Group A: Plans where the primary language is English
group_a_language = df[df['Language'] == 'English']

# Group B: Plans where the primary language is French
group_b_language = df[df['Language'] == 'French']


print(group_a_language, group_b_language)


         UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0                     145249     12827  2015-03-01 00:00:00             True   
1                     145249     12827  2015-05-01 00:00:00             True   
2                     145249     12827  2015-07-01 00:00:00             True   
3                     145255     12827  2015-05-01 00:00:00             True   
4                     145255     12827  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:00            False   

        Citizenship          LegalType 

In [29]:
# Working with VehicleType
# Group A: Plans for Sedan type vehicles
group_a_vehicle_type = df[df['VehicleType'] == 'Sedan']

# Group B: Plans for SUV type vehicles
group_b_vehicle_type = df[df['VehicleType'] == 'SUV']

print(group_a_vehicle_type, group_b_vehicle_type)


Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCode, MainCrestaZone, SubCrestaZone, ItemType, mmcode, VehicleType, RegistrationYear, make, Model, Cylinders, cubiccapacity, kilowatts, bodytype, NumberOfDoors, VehicleIntroDate, CustomValueEstimate, AlarmImmobiliser, TrackingDevice, CapitalOutstanding, NewVehicle, WrittenOff, Rebuilt, Converted, CrossBorder, NumberOfVehiclesInFleet, SumInsured, TermFrequency, CalculatedPremiumPerTerm, ExcessSelected, CoverCategory, CoverType, CoverGroup, Section, Product, StatutoryClass, StatutoryRiskType, TotalPremium, TotalClaims, LossRatio, ProfitMargin, ClaimFrequency]
Index: []

[0 rows x 55 columns] Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCod

In [30]:
# Working with CoverGroup
# Group A: Plans in Standard cover group
group_a_cover_group = df[df['CoverGroup'] == 'Standard']

# Group B: Plans in Premium cover group
group_b_cover_group = df[df['CoverGroup'] == 'Premium']

print(group_a_cover_group, group_b_cover_group)


Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCode, MainCrestaZone, SubCrestaZone, ItemType, mmcode, VehicleType, RegistrationYear, make, Model, Cylinders, cubiccapacity, kilowatts, bodytype, NumberOfDoors, VehicleIntroDate, CustomValueEstimate, AlarmImmobiliser, TrackingDevice, CapitalOutstanding, NewVehicle, WrittenOff, Rebuilt, Converted, CrossBorder, NumberOfVehiclesInFleet, SumInsured, TermFrequency, CalculatedPremiumPerTerm, ExcessSelected, CoverCategory, CoverType, CoverGroup, Section, Product, StatutoryClass, StatutoryRiskType, TotalPremium, TotalClaims, LossRatio, ProfitMargin, ClaimFrequency]
Index: []

[0 rows x 55 columns] Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCod

In [31]:
# Working with Product
# Group A: Plans with Basic product
group_a_product = df[df['Product'] == 'Basic']

# Group B: Plans with Advanced product
group_b_product = df[df['Product'] == 'Advanced']

print(group_a_product, group_b_product)



Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCode, MainCrestaZone, SubCrestaZone, ItemType, mmcode, VehicleType, RegistrationYear, make, Model, Cylinders, cubiccapacity, kilowatts, bodytype, NumberOfDoors, VehicleIntroDate, CustomValueEstimate, AlarmImmobiliser, TrackingDevice, CapitalOutstanding, NewVehicle, WrittenOff, Rebuilt, Converted, CrossBorder, NumberOfVehiclesInFleet, SumInsured, TermFrequency, CalculatedPremiumPerTerm, ExcessSelected, CoverCategory, CoverType, CoverGroup, Section, Product, StatutoryClass, StatutoryRiskType, TotalPremium, TotalClaims, LossRatio, ProfitMargin, ClaimFrequency]
Index: []

[0 rows x 55 columns] Empty DataFrame
Columns: [UnderwrittenCoverID, PolicyID, TransactionMonth, IsVATRegistered, Citizenship, LegalType, Title, Language, Bank, AccountType, MaritalStatus, Gender, Country, Province, PostalCod

## Statistical Testing

In [32]:
from statsmodels.stats.proportion import proportions_ztest



# Function to perform a t-test
def perform_t_test(group_a, group_b, metric):
    t_stat, p_value = stats.ttest_ind(group_a[metric], group_b[metric], equal_var=False)
    return t_stat, p_value

# Function to perform chi-squared test
def perform_chi_squared(group_a, group_b, column):
    contingency_table = pd.crosstab(group_a[column], group_b[column])
    
    # Check if the contingency table is empty
    if contingency_table.empty:
        raise ValueError("Contingency table is empty. Ensure both groups have data and the column is valid.")
    
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
    return chi2, p_value

# Function to print the results
def analyze_results(p_value):
    if p_value < 0.05:
        print("Reject the null hypothesis. There is a significant difference.")
    else:
        print("Fail to reject the null hypothesis. There is no significant difference.")



In [33]:
# Perform t-test for Loss Ratio between Group A and Group B for AlarmImmobiliser
t_stat, p_value = perform_t_test(group_a_alarm, group_b_alarm, 'LossRatio')

# Print the results
print(f"t-statistic: {t_stat}, p-value: {p_value}")
analyze_results(p_value)


t-statistic: nan, p-value: nan
Fail to reject the null hypothesis. There is no significant difference.


In [34]:
# Perform t-test for Profit Margin between Group A and Group B for TrackingDevice
t_stat, p_value = perform_t_test(group_a_tracking, group_b_tracking, 'ProfitMargin')

# Print the results
print(f"t-statistic: {t_stat}, p-value: {p_value}")
analyze_results(p_value)


t-statistic: nan, p-value: nan
Fail to reject the null hypothesis. There is no significant difference.


In [35]:
# Check if group_a_converted and group_b_converted are defined
if 'group_a_converted' in locals() and 'group_b_converted' in locals():
    # Print group sizes
    print(f"Group A size: {len(group_a_converted)}, Group B size: {len(group_b_converted)}")
    
    # Print 'Converted' column values
    if 'Converted' in group_a_converted.columns and 'Converted' in group_b_converted.columns:
        print("Group A 'Converted' column values:")
        print(group_a_converted['Converted'].value_counts())
        print("Group B 'Converted' column values:")
        print(group_b_converted['Converted'].value_counts())
    else:
        print("The 'Converted' column does not exist in one or both groups.")
    
    # Perform chi-squared test if both groups are not empty
    if not group_a_converted.empty and not group_b_converted.empty:
        chi2_stat, p_value = perform_chi_squared(group_a_converted, group_b_converted, 'Converted')
        
        # # Print the results
        print(f"Chi-squared statistic: {chi2_stat}, p-value: {p_value}")
        analyze_results(p_value)
    else:
        print("One or both groups are empty.")
else:
    print("Group A or Group B is not defined.")

Group A size: 358110, Group B size: 87
Group A 'Converted' column values:
Converted
No    358110
Name: count, dtype: int64
Group B 'Converted' column values:
Converted
Yes    87
Name: count, dtype: int64


ValueError: Contingency table is empty. Ensure both groups have data and the column is valid.

In [45]:
# Number of claims by gender
count_men = df[df['Gender'] == 'Male']['ClaimFrequency'].sum()
count_women = df[df['Gender'] == 'Female']['ClaimFrequency'].sum()

# Total number of policies by gender
nobs_men = df[df['Gender'] == 'Male'].shape[0]
nobs_women = df[df['Gender'] == 'Female'].shape[0]

# Perform a z-test for proportions
stat, p_value = proportions_ztest([count_men, count_women], [nobs_men, nobs_women])

# Print the results
print(f"z-statistic: {stat}, p-value: {p_value}")
analyze_results(p_value)


z-statistic: -0.7026689252186327, p-value: 0.48226209762294847
Fail to reject the null hypothesis. There is no significant difference.
